# Ansible

**Sources**

- [Building an inventory](https://docs.ansible.com/ansible/latest/getting_started/get_started_inventory.html).

## Setup

Using ansible requires:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. These containers require some preparation, so we'll create images.

For the managed node, we need to install and run the ssh server.

In [1]:
cat << EOF > ansible_files/managed_node_dockerfile

FROM python:3.10-alpine
RUN apk add -q openssh && ssh-keygen -A && mkdir /root/.ssh
CMD ["/usr/sbin/sshd", "-D"]

EOF

For control mode, we need to install the ssh client and ansible as a python package.

In [2]:
cat << EOF > ansible_files/control_node_dockerfile

FROM python:3.10-alpine
RUN apk add openssh-client && \\
    ssh-keygen -t rsa -N "" -f /root/.ssh/id_rsa && \\
    pip3 install ansible && \\
    # Add to the config statement that we don't need to enter if we want to save the new host.
    echo -e "Host *\n\tStrictHostKeyChecking no\n\n" > ~/.ssh/config

EOF

The following cell creates images that we'll use to consider ansible.

In [3]:
docker build -t managed_node \
    -f ./ansible_files/managed_node_dockerfile \
    ./ansible_files &> /dev/null
docker build -t control_node \
    -f ./ansible_files/control_node_dockerfile \
    ./ansible_files &> /dev/null

## Hello world

To make ansible work we need:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. The following cell creates docker containers and configures access between them.

In [4]:
docker network create ansible_test_network
docker run -itd --rm --name ansible_managed_node --network ansible_test_network managed_node
docker run -itd --rm --name ansible_control_node --network ansible_test_network control_node

export public_key=$(docker exec ansible_control_node cat /root/.ssh/id_rsa.pub)
docker exec ansible_managed_node sh -c "echo \"$public_key\" >> /root/.ssh/authorized_keys"

17b6dc4e2b5073f0fd96b6c2eba797671abce7f11bbe7f7f601a16b6b5bf5421
e2b60edf2f72ce4c188bc75984c723cf89a6011552f25ad363fb6108a5f65944
80a6c1ebd8efeca30d9a036c24965f2a8e81a33a7740239592aa12d28494f886


**Note** don't forget to stop containres after all.

In [9]:
docker stop ansible_managed_node ansible_control_node
docker network rm ansible_test_network

ansible_managed_node
ansible_control_node
ansible_test_network


### Inventory

Inventories organise managed nodes into centralised files that provide Ansible with system information and network locations. An inventory file allows Ansible to manage a large number of hosts with a single command. So it's a file in which you list all the managed nodes.

Now in the control node we need to create an ansible config `inventory.ini` where we just specify managed node.

In [5]:
docker exec ansible_control_node sh -c  "
cat << EOF > inventory.ini
[myhosts]
ansible_managed_node
EOF"

Here `[myhosts]` is the name of the group of servers.

Finally, try pinging the managed nodes using the command `ansible <group of servers> -m ping -i inventory.ini`.

In [6]:
docker exec ansible_control_node ansible myhosts -m ping -i inventory.ini

[WARNING]: Platform linux on host ansible_managed_node is using the discovered
Python interpreter at /usr/local/bin/python3.10, but future installation of
another Python interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.17/reference_appendices/interpreter_discovery.html for more information.
ansible_managed_node | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/local/bin/python3.10"
    },
    "changed": false,
    "ping": "pong"
}


As a result, we got the message `ansible_host | SUCCESS ...`, which tells us that we successfully ping the managed node.

### Playbook

Playbooks are files with instructions. So below is a playbook that creates a play called `My first play`. This play will ping all hosts and then print a message.

In [7]:
docker exec ansible_control_node sh -c  "
cat << EOF > playbook.yaml
- name: My first play
  hosts: myhosts
  tasks:
   - name: Ping my hosts
     ansible.builtin.ping:

   - name: Print message
     ansible.builtin.debug:
      msg: Hello world"

So now you can use the `ansible-playbook` command to play the playbook we have just created.

In [8]:
docker exec ansible_control_node ansible-playbook -i inventory.ini playbook.yaml


PLAY [My first play] ***********************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Platform linux on host ansible_managed_node is using the discovered
Python interpreter at /usr/local/bin/python3.10, but future installation of
another Python interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.17/reference_appendices/interpreter_discovery.html for more information.
ok: [ansible_managed_node]

TASK [Ping my hosts] ***********************************************************
ok: [ansible_managed_node]

TASK [Print message] ***********************************************************
ok: [ansible_managed_node] => {
    "msg": "Hello world"
}

PLAY RECAP *********************************************************************
ansible_managed_node       : ok=3    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



So there are tasks: `Ping my hosts` and `Print message`, just as we specified.